In [1]:

import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import XRPQuerier, XRPProcessor, XRPPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

from decoder5 import LogDecoder5
from decoder4 import LogDecoder
from config import Settings
from web3 import Web3



In [2]:
def decode_hex(value):
    """
    Decode a hexadecimal string to an integer if it's an Ethereum-style integer (e.g., block numbers, gas values).
    Does not decode long hashes or other non-integer hex values.
    :param value: Hexadecimal string (e.g., '0x677df92f') or other types.
    :return: Decoded integer or original value if not a valid short hex integer.
    """
    if isinstance(value, str) and value.startswith("0x"):
        if value == '0x':
            return None
        
        # Only decode if the hex string is short (e.g., block numbers, gas, timestamps)
        return int(value, 16)
    return value  # Return original value if not a short hex integer

# Parallelization of the tasks
def get_logs(block):
    
    w3 = Web3(Web3.HTTPProvider(Settings.ETHEREUM_ENDPOINT))
    
    # Fetch logs for the block
    logs = w3.eth.get_logs({
        "fromBlock": decode_hex(block['number']),
        "toBlock": decode_hex(block['number'])
    })

    return logs



In [3]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

queries = mongodb_query_ops.get_recent_blocks('Ethereum', 100)

query = mongodb_query_ops.get_block_by_number('Ethereum', 21585277)
data = query['raw_block_data']


2025-01-12 01:11:51,376 - INFO - Connected to MongoDB database: blockchain
2025-01-12 01:11:51,404 - INFO - Indexes created for collection: Solana
2025-01-12 01:11:51,406 - INFO - Indexes created for collection: Bitcoin
2025-01-12 01:11:51,407 - INFO - Indexes created for collection: Ethereum
2025-01-12 01:11:51,409 - INFO - Indexes created for collection: BNB
2025-01-12 01:11:51,410 - INFO - Indexes created for collection: XRP
2025-01-12 01:11:51,423 - INFO - Retrieved 100 most recent blocks from the Ethereum collection.
2025-01-12 01:11:51,585 - INFO - Retrieved block 21585277 from Ethereum collection in MongoDB.


Decoder Tests

In [6]:
# Initialize with database path
decoder = LogDecoder5(
    etherscan_api_key=Settings.ETHERSCAN_API_KEY,
    db_path="events.db"
)

decoded_logs = {}
for block in queries:
    logs = get_logs(block['raw_block_data'])
    for i in range(2):
        decoded = decoder.process_transaction_logs(logs)
        decoded_logs.update(decoded)

print(len(decoded_logs))



10623


In [8]:
decoded_logs

{'0xe1d2a4f5c548caf73f18034a07a3f364508a0cffcb1e86d1ce77156b89097c8b': [{'event': 'Transfer',
   'param1': '0x398f911721921d1143e38845d787afb615531a5e',
   'param2': '0x7ec5206a9b84ea09cfc07548079f781709d7844a',
   'param3': 200000000000000,
   'contract': '0x283D480dFD6921055E9C335FC177bF8Cb9C94184'},
  {'event': 'Approval',
   'param1': '0x398f911721921d1143e38845d787afb615531a5e',
   'param2': '0x000000000022d473030f116ddee9f6b43ac78ba3',
   'param3': 2661965793883000,
   'contract': '0x283D480dFD6921055E9C335FC177bF8Cb9C94184'},
  {'event': 'Transfer',
   'param1': '0x7ec5206a9b84ea09cfc07548079f781709d7844a',
   'param2': '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad',
   'param3': 900132921790335159,
   'contract': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'},
  {'event': 'Unknown',
   'signature': '1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1',
   'topics': [],
   'data_chunks': [HexBytes('0x00000000000000000000000000000000000000000000001313afe5de1f07000000000

In [4]:
from decoder7 import LogDecoder3
decoder = LogDecoder3(
    etherscan_api_key=Settings.ETHERSCAN_API_KEY,
    db_path="events.db"
)

In [5]:
decoded_logs1 = {}
for block in queries[:10]:
    logs = get_logs(block['raw_block_data'])
    results = decoder.process_transaction_logs(logs, store_undecoded=True)
    decoded_logs1.update(results)

print(len(decoded_logs))



Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: Object of type HexBytes is not JSON serializable
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 'transactionHash'
Error storing undecoded log: 't

KeyboardInterrupt: 

In [9]:
decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)

decoded_logs1 = {}
for block in queries:
    logs = get_logs(block['raw_block_data'])
    for i in range(5):
        decoded = decoder.process_transaction_logs(logs)
        decoded_logs1.update(decoded)

print(len(decoded_logs1))



In [23]:
from decoder3 import LogDecoder3

decoder = LogDecoder3(Settings.ETHERSCAN_API_KEY)

decoded_logs2 = {}
for block in queries:
    logs = get_logs(block['raw_block_data'])
    for i in range(1):
        decoded = decoder.process_transaction_logs(logs)
        decoded_logs2.update(decoded)

print(len(decoded_logs2))

